#Importing Libraries and Data Prep

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras import layers
import scipy as sc
from scipy import signal

In [ ]:
tensorflow.config.run_functions_eagerly(True)

In [ ]:
!nvidia-smi

Sat Sep  3 15:14:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |    286MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df = pd.read_csv(r"/content/50_Users_Acceleromter_Data.csv")

In [ ]:
length = 15000

In [ ]:
import time
start_time = time.time()
N_Users=25
k=0
X=[0 for p in range(N_Users)] # Final Array
for i in range(0,length*N_Users,length):
    temp=[]
    for j in range(length):
        x_y_z = [df.iloc[i+j,0],df.iloc[i+j,1],df.iloc[i+j,2]]
        temp.append(x_y_z)
    X[k]=temp
    k+=1 #incrementing [Final Array] indices
print("--- %s seconds ---" % (time.time() - start_time))

--- 29.181073665618896 seconds ---


In [ ]:
t=np.array(X)
t.shape

(25, 15000, 3)

In [ ]:
# Main Algo for Creating Windows .....#1

def create(df,wind_size,hop_size,m_index):
  #l=((len(df.x)-wind_size)/hop_size)+1
  #rows, cols = (wind_size, 3) 
#  a = [[0 for i in range(cols)] for j in range(rows)] 
  array= [0 for i in range(int((length-wind_size)/hop_size)+1)]
  k = 0
  for i in range(0,(length-wind_size+1),hop_size):
      a=create1(df,i,wind_size,m_index)
       #array.append(a)
      array[k] = a
      k+=1
  #return pd.DataFrame(array)
  return array


In [ ]:
# Main Algo for Creating Windows .....#2

def create1(df,x,w,m_index):
     rows = w 
     cols = 9
     arr = [[0 for i in range(cols)] for j in range(rows)] 
     for i in range(0,w,1):
        arr[i][0]=df[x+i][0]
        arr[i][1]=df[x+i][1]
        arr[i][2]=df[x+i][2]
     arr = np.array(arr)
     #for i in range(0,w,1):
        #arr[i][3]=np.mean(arr,axis=0)[0]
        #arr[i][4]=np.mean(arr,axis=0)[1]
        #arr[i][5]=np.mean(arr,axis=0)[2]
        #arr[i][6]=np.std(arr,axis=0)[0]
        #arr[i][7]=np.std(arr,axis=0)[1]
        #arr[i][8]=np.std(arr,axis=0)[2]
        #arr[i][9]=abs(sc.fft(arr[:,0]))[i]
        #arr[i][10]=abs(sc.fft(arr[:,1]))[i]
        #arr[i][11]=abs(sc.fft(arr[:,2]))[i]
     f1,t1,x = signal.stft(arr[:,0],5,nperseg=10)
     f2,t2,y = signal.stft(arr[:,1],5,nperseg=10)
     f3,t3,z = signal.stft(arr[:,2],5,nperseg=10)
     temp_a = np.concatenate((abs(np.array(x)),abs(np.array(y)),abs(np.array(z))),axis=0)
     return temp_a

In [ ]:
import time
start_time = time.time()
X_final = [0 for i in range( N_Users)]
for m_index in range(N_Users):
  X_final[m_index]=create(X[m_index],225,10,m_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 44.10889768600464 seconds ---


In [ ]:
t=np.array(X_final[0])
t.shape             # The shape gives(number of windows,shape of TF plot image)

(1478, 18, 46)

In [ ]:
X_input=[]
for i in range(N_Users):
  temp = X_final[i]
  X_input += temp
X_input = np.array(X_input)

In [ ]:
y = [0 for i in range(X_input.shape[0])]
for i in range(X_input.shape[0]):
  y[i]=int(i*N_Users/X_input.shape[0])
y = np.array(y)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_input,y,test_size = 0.2,shuffle=True)

In [ ]:
#Reshaping the input
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [ ]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout

#VGG16

In [ ]:
clf = tensorflow.keras.Sequential()

In [ ]:

clf.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.MaxPool2D(pool_size=(1,1),strides = 1))
clf.add(keras.layers.Dropout(rate= 0.5))

clf.add(keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding="same", strides = 1,activation="relu"))
clf.add(keras.layers.MaxPool2D(pool_size=(1,1),strides = 1))
clf.add(keras.layers.Dropout(rate= 0.5))

clf.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", strides = 1,activation="relu"))
clf.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.MaxPool2D(pool_size=(1,1),strides = 1))
clf.add(keras.layers.Dropout(rate= 0.5))

clf.add(keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", strides = 1,activation="relu"))
clf.add(keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", strides = 1,activation="relu"))
clf.add(keras.layers.MaxPool2D(pool_size=(1,1),strides = 1))
clf.add(keras.layers.Dropout(rate= 0.5))

clf.add(keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same",strides = 1, activation="relu"))
clf.add(keras.layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", strides = 1,activation="relu"))
clf.add(keras.layers.MaxPool2D(pool_size=(1,1),strides = 1))
clf.add(keras.layers.Dropout(rate= 0.5))
                    
clf.add(keras.layers.Flatten())
                    
clf.add(keras.layers.Dense(128, activation="relu"))
clf.add(keras.layers.Dense(128, activation='relu'))
clf.add(keras.layers.Dense(N_Users, activation='softmax'))        

In [ ]:
clf.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0005493824470636434),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = clf.fit(X_train,y_train,batch_size=64,epochs=50,validation_data=(X_test,y_test))

79/93 [========================>.....] - ETA: 7s - loss: 1.6094 - accuracy: 0.1984

KeyboardInterrupt: ignored

#AlexNET

In [ ]:
N_Users = 25
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu',),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(N_Users,activation='softmax')  
    
    
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tensorflow.optimizers.SGD(lr=0.001), metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 2, 9, 128)         15616     
                                                                 
 batch_normalization_15 (Bat  (None, 2, 9, 128)        512       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 2, 9, 128)        0         
 2D)                                                             
                                                                 
 conv2d_16 (Conv2D)          (None, 2, 9, 256)         819456    
                                                                 
 batch_normalization_16 (Bat  (None, 2, 9, 256)        1024      
 chNormalization)                                                
                                                      

In [ ]:
history = model.fit(X_train,y_train,batch_size=128,epochs=1,validation_data=(X_test,y_test))

231/231 [==============================] - 10s 43ms/step - loss: 3.2367 - accuracy: 0.1318 - val_loss: 3.0383 - val_accuracy: 0.1104


In [ ]:
from sklearn import metrics
pred_y = model.predict(X_test)
y_pred=[]
for i in range(y_test.shape[0]):
     temp=np.array(pred_y[i])    
     #maximum = np.where(temp==max(temp))
     y_pred.append(np.argmax(temp)) 
recall = metrics.recall_score(y_test, y_pred, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test, y_pred, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

#LeNet

In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu',))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=N_Users, activation = 'softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 16, 44, 6)         60        
                                                                 
 average_pooling2d (AverageP  (None, 8, 22, 6)         0         
 ooling2D)                                                       
                                                                 
 conv2d_21 (Conv2D)          (None, 6, 20, 16)         880       
                                                                 
 average_pooling2d_1 (Averag  (None, 3, 10, 16)        0         
 ePooling2D)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 480)               0         
                                                                 
 dense_11 (Dense)            (None, 120)              

In [ ]:
history = model.fit(X_train,y_train,batch_size=128,epochs=1,validation_data=(X_test,y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


231/231 [==============================] - 7s 28ms/step - loss: 2.7290 - accuracy: 0.1779 - val_loss: 2.4473 - val_accuracy: 0.2503


In [ ]:
from sklearn import metrics
pred_y = model.predict(X_test)
y_pred=[]
for i in range(y_test.shape[0]):
     temp=np.array(pred_y[i])    
     #maximum = np.where(temp==max(temp))
     y_pred.append(np.argmax(temp)) 
recall = metrics.recall_score(y_test, y_pred, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test, y_pred, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.8104 Precision: 0.7753 F-Measure: 0.7925


#ZFNet

In [ ]:
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(7,7), strides=(2,2), activation='relu',),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(1,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(N_Users,activation='softmax')])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tensorflow.optimizers.SGD(lr=0.001), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 6, 20, 96)         4800      
                                                                 
 batch_normalization_20 (Bat  (None, 6, 20, 96)        384       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 6, 20, 96)        0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 6, 20, 256)        221440    
                                                                 
 batch_normalization_21 (Bat  (None, 6, 20, 256)       1024      
 chNormalization)                                                
                                                      

In [ ]:
history = model.fit(X_train,y_train,batch_size=128,epochs=1,validation_data=(X_test,y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


231/231 [==============================] - 26s 107ms/step - loss: 3.1536 - accuracy: 0.1561 - val_loss: 3.0482 - val_accuracy: 0.1116


In [ ]:
from sklearn import metrics
pred_y = model.predict(X_test)
y_pred=[]
for i in range(y_test.shape[0]):
     temp=np.array(pred_y[i])    
     #maximum = np.where(temp==max(temp))
     y_pred.append(np.argmax(temp)) 
recall = metrics.recall_score(y_test, y_pred, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test, y_pred, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.6846 Precision: 0.6986 F-Measure: 0.6915
